In [1]:
import json
import pandas as pd
import numpy as np
import re

In [2]:
file_dir ='C:/Users/vick_/Desktop/Data Analytics Projects/Movies-ETL'

In [3]:
movies_json=f'{file_dir}/wikipedia.movies.json'

In [4]:
def wiki(movies_json):
    with open(movies_json, mode='r') as file:
        wiki_movies_raw = json.load(file)

In [5]:
    wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

NameError: name 'wiki_movies_raw' is not defined

In [ ]:
    def clean_movie(movie):
            movie = dict(movie) #create a non-destructive copy
            alt_titles = {}
            # combine alternate titles into one list
            for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                        'Hangul','Hebrew','Hepburn','Japanese','Literally',
                        'Mandarin','McCune-Reischauer','Original title','Polish',
                        'Revised Romanization','Romanized','Russian',
                        'Simplified','Traditional','Yiddish']:
                if key in movie:
                    alt_titles[key] = movie[key]
                    movie.pop(key)
            if len(alt_titles) > 0:
                movie['alt_titles'] = alt_titles

            # merge column names
            try:
                def change_column_name(old_name, new_name):
                    if old_name in movie:
                        movie[new_name] = movie.pop(old_name)
                        change_column_name('Adaptation by', 'Writer(s)')
                        change_column_name('Country of origin', 'Country')
                        change_column_name('Directed by', 'Director')
                        change_column_name('Distributed by', 'Distributor')
                        change_column_name('Edited by', 'Editor(s)')
                        change_column_name('Length', 'Running time')
                        change_column_name('Original release', 'Release date')
                        change_column_name('Music by', 'Composer(s)')
                        change_column_name('Produced by', 'Producer(s)')
                        change_column_name('Producer', 'Producer(s)')
                        change_column_name('Productioncompanies ', 'Production company(s)')
                        change_column_name('Productioncompany ', 'Production company(s)')
                        change_column_name('Released', 'Release Date')
                        change_column_name('Release Date', 'Release date')
                        change_column_name('Screen story by', 'Writer(s)')
                        change_column_name('Screenplay by', 'Writer(s)')
                        change_column_name('Story by', 'Writer(s)')
                        change_column_name('Theme music composer', 'Composer(s)')
                        change_column_name('Written by', 'Writer(s)')
            except:
                print('check error-for dollar converstion')

            return movie


In [ ]:
    clean_movies = [clean_movie(movie) for movie in wiki_movies]
    wiki_movies_df = pd.DataFrame(clean_movies)

In [ ]:
    # STEP 1 REMOVE DUPLICATE ROWS

    wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')

    wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)

    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]

    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

In [ ]:
    # STEP 2: CHANGING DATA TYPE - drop null rows
    box_office = wiki_movies_df['Box office'].dropna() 
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
    # Step 3: Change dollar signs
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
    box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [ ]:
    matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE)
    matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE)

In [ ]:
    def parse_dollars(s):
    # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a million
            value = float(s) * 10**6

            # return value
            return value

        # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a billion
            value = float(s) * 10**9

            # return value
            return value

        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            # remove dollar sign and commas
            s = re.sub('\$|,','', s)

            # convert to float
            value = float(s)

            # return value
            return value

        # otherwise, return NaN
        else:
            return np.nan

In [ ]:
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars) 

In [ ]:
    #Step 4: Parsing Budget
    #1 Create a budget variable with the following code:
    budget = wiki_movies_df['Budget'].dropna()
    #2 Convert to String (Ask andrew and tutor)
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
     # Step 3 Remove dollar sign and a hypen
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    # 4 Match form one and two from the first function
    matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE)
    matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE)
    budget = budget.str.replace(r'\[\d+\]\s*', '')
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Budget', axis=1, inplace=True)

In [ ]:
    #Step 5: Parsing Date
    #1  drop Null rows
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    # 2 create variables
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
     #3 release date
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

In [ ]:
    # STEP 6 PARSE RUNTIME
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
     running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    # dataframe is all string convert it to numeric values and Nan values fill to zero
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

    
    
    # Convert the hour group to  minutes
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)

In [ ]:
    #2 

In [ ]:
kaggle_metadata.tail()

In [ ]:
wiki_movies_df=pd.DataFrame(wiki_movies_raw)
wiki_movies_df.head()

In [ ]:
wiki_movies_df.columns

In [ ]:
wiki_movies_df.columns.tolist()

In [ ]:
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

In [ ]:
wiki_movie_df=pd.DataFrame(wiki_movies)
wiki_movie_df.head()

In [ ]:
# Lambda functions
#square=lambda x: x*x
#square(5)

In [ ]:
# Global and local functions 

#x = 'global value'

#def foo():
   # x = 'local value'
    #print(x)

#foo()
#print(x)

In [ ]:
# copy list
    #new_list=list(old_list)
# copy dictionary 
    # new_dict=dict(old_dict)

In [ ]:
def clean_movie(movie):
    movie=dict(movie) # create a non-destructive copy
    return movie


In [ ]:
wiki_movie_df[wiki_movie_df['Arabic'].notnull()]

In [ ]:
wiki_movie_df[wiki_movie_df['Arabic'].notnull()]['url']

In [ ]:
wiki_movie_df.info()

In [ ]:
# Ask how to get unique values to look up what's in list value_counts()
#(wiki_movie_df['Arabic'].tolist())
type(wiki_movie_df['Arabic'])
wiki_movie_df['Mandarin'].value_counts()

In [ ]:
# function to conslidate values from multiple columns to one column
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    # Step 1: Make an empty dict to hold all of the alternative titles.
    alt_titles = {}
    # Step 2: Loop through a list of all alternative title keys.
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        #Step 2a: Check if the current key exists in the movie object.
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
        # Step 2b: If so, remove the key-value pair and add to the alternative titles dictionary.   
        
    # Step 3: After looping through every key, add the alternative titles dict to the movie object.
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    
    return movie

In [ ]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [ ]:
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

# Consolidate Mulitple Columns into One and Remame Columns including consolidation

In [ ]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie


In [ ]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]
wiki_movies_df = pd.DataFrame(clean_movies)

In [ ]:
sorted(wiki_movies_df.columns.tolist())

# Remove duplicate Rows

In [ ]:
#Pandas’ built-in string methods that work on a 
#Series object accessed with the str property. 
#We’ll be using str.extract(), which takes in a regular expression pattern. 
# IMDb links generally look like “https://www.imdb.com/title/tt1234567/,
#” with “tt1234567” as the IMDb ID. The regular expression for a group
# of characters that start with “tt” and has seven digits is "(tt\d{7})". 

wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
# Inplace True is like wiki_df=wiki_df.drop_duplicate
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
wiki_movies_df.head()

In [ ]:
[[column,wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]

In [ ]:
len(wiki_movies_df)

In [ ]:
# Removing columns where 90% of values is null

wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

In [ ]:
wiki_movies_df.info()

# Making Dataset to convert to perticular data-type so that it can be imported in SQL; Parse the data

In [ ]:
wiki_movies_df.dtypes

# Changing data types
Box office should be numeric.
Budget should be numeric.
Release date should be a date object.
Running time should be numeric.

# Parsing Dollar

In [ ]:
box_office = wiki_movies_df['Box office'].dropna() 

In [ ]:
len(box_office)

In [ ]:
# Function - Not a String

def is_not_a_string(x):
    return type(x) != str

In [ ]:
#apply box_office Not string
box_office[box_office.map(is_not_a_string)]


In [ ]:
# Lamda function of not a string

lambda x: type(x) != str
# box_office[box_office.map(lambda x: type(x) != str)]

In [ ]:
some_list = ['One','Two','Three']
'Mississippi'.join(some_list)

In [ ]:
# Ask Andrew about this lambda function and Join

box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

Step 2: Add an arbitrary (but non-zero) number of digits.
We’ll add the \d character to specify digits only, and the + modifier to capture one or more digits. Our regular expression string now appears as "\$\d+".
Step 3: Add an optional decimal point.
Remember, the decimal point is a special character, so it needs to be escaped with a backslash. Since the decimal point is optional, add a question mark modifier after it. Our regular expression string now appears as "\$\d+\.?".
Step 4: Add an arbitrary (but possibly zero) number of more digits.
Once again, we’ll use the \d character to specify digits only, but now with the * modifier because there may be no more digits after the decimal point. Our regular expression string now appears as "\$\d+\.?\d*".
Step 5: Add a space (maybe more than one).
Now we’re going to use the \s character to match whitespace characters. To be safe, we’ll match any number of whitespace characters with the * modifier. Our regular expression string now appears as "\$\d+\.?\d*\s*".
Step 6: Add the word “million” or “billion.”
Since “million” and “billion” only differ by one letter, we can match it with a character set for the first letter. We specify character sets with square brackets, so we’ll add "[mb]illion" to the end of our string. Our finished regular expression string now appears as "\$\d+\.?\d*\s*[mb]illion".
Create a variable form_one and set it equal to the finished regular expression string. Because we need the escape characters to remain, we need to preface the string with an r.
form_one = r'\$\d+\.?\d*\s*[mb]illion'

In [ ]:
form_one = r'\$\d+\.?\d*\s*[mb]illion'

In [ ]:
box_office.str.contains(form_one, flags=re.IGNORECASE).sum()

In [ ]:
# form: $123,456,789.”
form_two = r'\$\d{1,3}(?:,\d{3})+'
box_office.str.contains(form_two, flags=re.IGNORECASE).sum()

In [ ]:
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE)

Instead, Pandas has element-wise logical operators:
The element-wise negation operator is the tilde: ~ (similar to “not”)
The element-wise logical “and” is the ampersand: &
The element-wise logical “or” is the pipe: |

In [ ]:
box_office[~matches_form_one & ~matches_form_two]

In [ ]:
form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

Psuedo Code - dollar function
def parse_dollars(s):
    # if s is not a string, return NaN
    # if input is of the form $###.# million
        # remove dollar sign and " million"
        # convert to float and multiply by a million
        # return value
    # if input is of the form $###.# billion
        # remove dollar sign and " billion"
        # convert to float and multiply by a billion
        # return value
    # if input is of the form $###,###,###
        # remove dollar sign and commas
        # convert to float
        # retrn value

    # otherwise, return NaN

use re.match(pattern, string) to see if our string matches a pattern. 
use re.sub(pattern, replacement_string, string) to remove dollar signs, spaces, commas, and letters, if necessary.

In [ ]:
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,','', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [ ]:
wiki_movies_df['box_office'] = box_offiMce.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

# Parsing Budget

In [ ]:
#1 Create a budget variable with the following code:
budget = wiki_movies_df['Budget'].dropna()


In [ ]:
budget

In [ ]:
# Step 2 Convert to String (Ask andrew and tutor)

budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
# Step 3 Remove dollar sign and a hypen
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [ ]:
budget


In [ ]:
# Step 4 Match form one and two from the first function
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE)
# Step 5 Show what does not match
budget[~matches_form_one & ~matches_form_two]

In [ ]:
budget = budget.str.replace(r'\[\d+\]\s*', '')
budget[~matches_form_one & ~matches_form_two]

Everything is now ready to parse the budget values. We can copy the line of code we used to parse the box office values, changing “box_office” to “budget”:

In [6]:
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

NameError: name 'budget' is not defined

In [ ]:
wiki_movies_df.drop('Budget', axis=1, inplace=True)

# Parse Release Date

The forms we’ll be parsing are:
Full month name, one- to two-digit day, four-digit year (i.e., January 1, 2000)
Four-digit year, two-digit month, two-digit day, with any separator (i.e., 2000-01-01)
Full month name, four-digit year (i.e., January 2000)
Four-digit year

In [ ]:
release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
date_form_two = r'\d{4}.[01]\d.[123]\d'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'

In [ ]:
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

In [ ]:
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)


Parse Running Time

In [ ]:
running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE).sum()

In [ ]:
running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE) != True]

In [ ]:
running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE).sum()

In [ ]:
running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE) != True]

In [ ]:
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

 this new DataFrame is all strings, we’ll need to convert them to numeric values. Because we may have captured empty strings, we’ll use the to_numeric() method and set the errors argument to 'coerce'. Coercing the errors will turn the empty strings into Not a Number (NaN), then we can use fillna() to change all the NaNs to zeros.

In [ ]:
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

convert the hour capture groups and minute capture groups to minutes if the pure minutes capture group is zero, and save the output to wiki_movies_df:

In [ ]:
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

Finally, we can drop Running time from the dataset with the following code:

In [ ]:
wiki_movies_df.drop('Running time', axis=1, inplace=True)

# Clean the Kaggle Data

In [ ]:
kaggle_metadata.dtypes

In [ ]:
kaggle_metadata['adult'].value_counts()

In [7]:
# Remove Bad Data
kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]

NameError: name 'kaggle_metadata' is not defined

In [ ]:
kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')

Video Column

In [ ]:
kaggle_metadata['video'].value_counts()

In [ ]:
kaggle_metadata['video'] == 'True'

In [ ]:
kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'

In [ ]:
#For the numeric columns, we can just use the to_numeric() method 
#from Pandas. We’ll make sure the errors= argument is set to 'raise',
#so we’ll know if there’s any data that can’t be converted to numbers.

kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')

In [ ]:
kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

# Load and Inspect Rating.info

In [ ]:
ratings.info(null_counts=True)

Storing time values as a data type is difficult, and there are many, many standards out there for time values. Some store time values as text strings, like the ISO format “1955-11-05T12:00:00,” but then calculating the difference between two time values is complicated and computationally expensive. The Unix time standard stores points of time as integers, specifically as the number of seconds that have elapsed since midnight of January 1, 1970. This is known as the Unix epoch. There are other epochs in use, but the Unix epoch is by far the most widespread.

In [ ]:
pd.to_datetime(ratings['timestamp'], unit='s')

In [ ]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

A histogram is a bar chart that displays how often a data point shows up in the data. A histogram is a quick, visual way to get a sense of how a dataset is distributed.

In [ ]:
ratings['rating'].describe()

In [ ]:
ratings['rating'].plot(kind='hist')

# 8.4.1 Merging Wikipedia and Kaggle Matadata

In [8]:
movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])

NameError: name 'wiki_movies_df' is not defined

In [ ]:
movies_df.info()

In [ ]:
# Competing data:
# Wiki                     Movielens                Resolution
#--------------------------------------------------------------------------
# title_wiki               title_kaggle
# running_time             runtime
# budget_wiki              budget_kaggle
# box_office               revenue
# release_date_wiki        release_date_kaggle
# Language                 original_language
# Production company(s)    production_companies  

In [ ]:
movies_df[['title_wiki','title_kaggle']]

In [ ]:
#look at the rows where the titles don’t match.
movies_df[movies_df['title_wiki'] != movies_df['title_kaggle']][['title_wiki','title_kaggle']]

In [ ]:
movies_df[(movies_df['title_kaggle'] == '') | (movies_df['title_kaggle'].isnull())]

A scatter plot is a great way to give us a sense of how similar the columns are to each other. If the two columns were exactly the same, we’d see a scatter plot of a perfectly straight line. Any wildly different values will show up as dots far from that central line, and if one column is missing data, those values will fall on the x-axis or y-axis.

In [ ]:
#rged data, we should expect there to be missing values.
# Scatter plots won’t show null values, so we need to 
#fill them in with zeros when we’re making our plots to get the whole picture.

movies_df.fillna(0).plot(x='running_time', y='runtime', kind='scatter')

In [ ]:
# budget_wiki and budget_kaggle are numeric, we’ll make another scatter
# plot to compare the values:

movies_df.fillna(0).plot(x='budget_wiki',y='budget_kaggle', kind='scatter')

In [ ]:
#The box_office and revenue columns are numeric, 
# so we’ll make another scatter plot.

movies_df.fillna(0)[movies_df['box_office'] < 10**9].plot(x='box_office', y='revenue', kind='scatter')

For release_date_wiki and release_date_kaggle, we can’t directly make a scatter plot, because the scatter plot only works on numeric data. However, there’s a tricky workaround that we can use. We’ll use the regular line plot (which can plot date data), and change the style to only put dots by adding style='.' to the plot() method:

In [ ]:
movies_df[['release_date_wiki','release_date_kaggle']].plot(x='release_date_wiki', y='release_date_kaggle', style='.')

In [ ]:
movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')]

In [ ]:
movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')].index

In [ ]:
movies_df = movies_df.drop(movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')].index)

In [ ]:
movies_df[movies_df['release_date_wiki'].isnull()]

In [ ]:
movies_df['Language'].value_counts()

In [9]:
movies_df['Language'].apply(lambda x: tuple(x) if type(x) == list else x).value_counts(dropna=False)

NameError: name 'movies_df' is not defined

In [ ]:
movies_df['original_language'].value_counts(dropna=False)

In [ ]:
# Production Companies

movies_df[['Production company(s)','production_companies']]

In [ ]:
#Step 1) First, we’ll drop the title_wiki,\
# release_date_wiki, Language, and Production company(s) columns.

movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)


In [ ]:
#  little time, we’ll make a function that fills in missing data for a column pair 
# and then drops the redundant column.

def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
    df[kaggle_column] = df.apply(
        lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
        , axis=1)
    df.drop(columns=wiki_column, inplace=True)

In [ ]:
# n the function for the three column pairs that we decided to fill in zeros.

fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
movies_df

 good to check that there aren’t any columns with only one value, since that doesn’t really provide any information. Don’t forget, we need to convert lists to tuples for value_counts() to work.

In [ ]:
for col in movies_df.columns:
    lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
    value_counts = movies_df[col].apply(lists_to_tuples).value_counts(dropna=False)
    num_values = len(value_counts)
    if num_values == 1:
        print(col)

In [ ]:
movies_df['video'].value_counts(dropna=False)

We should reorder the columns to make the dataset easier to read for the hackathon participants. Having similar columns near each other helps people looking through the data get a better sense of what information is available. One way to reorder them would be to consider the columns roughly in groups, like this:
Identifying information (IDs, titles, URLs, etc.)
Quantitative facts (runtime, budget, revenue, etc.)
Qualitative facts (genres, languages, country, etc.)
Business data (production companies, distributors, etc.)
People (producers, director, cast, writers, etc.)

In [ ]:
movies_df = movies_df[['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages','Country',
                       'production_companies','production_countries','Distributor',
                       'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                      ]]

In [ ]:
# Rename Columns

movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)

In [ ]:
movies_df.head()

# 8.4.2 Transform and Merge Rating Data


Britta wants to include the rating data with the movie data, but it’s a very large dataset. She wants to reduce the ratings data to a useful summary of rating information for each movie, and then make the full dataset available to the hackathon participants if they decide they need more granular rating information.

In [ ]:
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count()

In [10]:
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1) 

NameError: name 'ratings' is not defined

 pivot this data so that movieId is the index, the columns will be all the rating values, and the rows will be the counts for each rating value.

In [ ]:
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1) \
                .pivot(index='movieId',columns='rating', values='count')

In [ ]:
rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]

In [ ]:
# This time, we need to use a left merge, since we want to keep everything in movies_df:

movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')

In [ ]:
# Finally, because not every movie got a rating for each rating level, there will be missing values instead of zeros. We have to fill those in ourselves, like this:

movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)